In [56]:
import config.env

In [57]:
# import os
# 
# os.environ["LANGSMITH_TRACING"] = "true"

In [58]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

save_local = "save_local"

db = FAISS.load_local(save_local, OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [59]:
# query = "MacBook Air15寸的重量"

query = "MacBook Air15寸的尺寸"

# query = "MacBook Air16寸的尺寸"

In [60]:
## 当向量数据库中没有合适答案时，使用大语言模型能力

In [61]:
from langchain.globals import set_debug,set_verbose

import logging

logging.basicConfig(filename="log.log")

set_verbose(True) # 没有更多的信息
set_debug(True)  # 全部的信息，太多了

In [62]:
def format_docs(docs):
    return "\n\n".join(doc.page_content  for doc in docs)

In [63]:
from langchain_core.prompts import ChatPromptTemplate

template = """
# 角色
你是一位专业且极具洞察力的商品问答专家，
拥有精准的眼光和丰富的经验，能够迅速且透彻地理解用户的问题，
在丰富的知识库中高效、准确地搜索有用信息，
为用户提供详尽、精确且易于理解的答案。

## 技能
### 技能 1: 深度剖析与精确搜索
1. 全方位、深层次地剖析用户提出的问题，精准提取关键核心要点。
2. 依据关键要点，在知识库中进行全面、深入且精确的检索。

### 技能 2: 细致对比商品差异
1. 依据检索获取的商品信息，梳理并对比每件商品之间的信息差别、区别，把每项结果记录下来。例如：都有描述屏幕大小，一个是6寸，一个是5.5寸，这就是两者的差异、区别。把结果存起来，为后面的回复做准备
2. 若商品不存在差异点，则无需记录。

### 技能 3: 精心组织回复
1. 将技能 2得出的差异结果，以清晰、有条理的表格形式展示。
2. 为用户生成确切、简洁且通俗易懂的回答。

## 约束
- 如果检索结果没有数据，请如实回答没有数据，不要自作主张填充或修改数据。
- 只回答与产品有关的问题，坚决不涉及无关内容。
- 始终使用清晰、简洁、通俗易懂的语言回应用户的问题。

#
Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_messages(
    ["system",template,("user", "{question}")],
)

In [64]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-ca",)

In [65]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

qa_chain = (
    {
        "context": db.as_retriever(  search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}) | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [66]:
qa_chain.invoke(query)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "MacBook Air15寸的尺寸"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "MacBook Air15寸的尺寸"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "MacBook Air15寸的尺寸"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "MacBook Air15寸的尺寸"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "MacBook Air15寸的尺寸"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > chain:format_docs] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:RunnableP

'很抱歉，当前提供的信息中没有包含MacBook Air 15寸的具体尺寸数据。您可以访问苹果官网或联系苹果客服获取最新的产品规格和详细信息。'

## 总结

1. chunk_size不能设100，搜索的结果不一定是当前需要的信息
例如：vector库入面存在13寸和15寸MBA的价格，按照上面的规则进行搜索，当搜索出价格就会停止（不管是谁的价格）

错误答案：
13英寸的MacBook Air有几种配置和价格选项：\n\n1. **RMB 10,499** - 这可能是较低配置的价格，可能包括较低的存储容量和处理器选项。\n   \n2. **RMB 11,999** - 这可能是中等配置的价格，可能包括更大的存储容量或者更高性能的处理器。\n\n3. **RMB 13,499** - 这可能是高端配置

原因：
13寸的分段
[Document(page_content='# MacBook Air13寸\n\n## 外观\n\n银色 星光色 深空灰色 午夜色'),
 Document(page_content='价格\n\nRMB 7999 (8 核图形处理器，256GB 固态硬盘)\nRMB 9499 (10 核图形处理器，512GB 固态硬盘)\nRMB 8999\nRMB 10,499\nRMB 11,999'),
 
15寸的分段：
Document(page_content='# MacBook Air15寸\n\n## 外观\n\n银色、星光色、深空灰色、午夜色\n\n## 价格\n\nRMB 10,499\nRMB 11,999\nRMB 13,499'),


2. 如果数据库搜索不出来结果，而ai又返回了数据，说明ai产生幻觉